<a href="https://colab.research.google.com/github/WookwonShim/data-analysis/blob/main/2018_central_park_squirrel_census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading and Preparation

In [5]:
from google.colab import drive
import pandas as pd
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Load data
squirrel = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/2018-central-park-squirrel-census-squirrel-data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Exploration

In [6]:
squirrel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 31 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   X                                           3023 non-null   float64
 1   Y                                           3023 non-null   float64
 2   Unique Squirrel ID                          3023 non-null   object 
 3   Hectare                                     3023 non-null   object 
 4   Shift                                       3023 non-null   object 
 5   Date                                        3023 non-null   int64  
 6   Hectare Squirrel Number                     3023 non-null   int64  
 7   Age                                         2902 non-null   object 
 8   Primary Fur Color                           2968 non-null   object 
 9   Highlight Fur Color                         1937 non-null   object 
 10  Combination 

In [7]:
squirrel.sample(n=5, random_state=28).T

,1945,1174,984,663,1757
X,-73.974718,-73.955326,-73.968172,-73.956366,-73.960156
Y,40.769282,40.797559,40.78356,40.796682,40.790051
Unique Squirrel ID,5E-PM-1012-05,40D-PM-1013-04,22B-PM-1014-07,39D-AM-1008-03,31E-PM-1006-08
Hectare,05E,40D,22B,39D,31E
Shift,PM,PM,PM,AM,PM
Date,10122018,10132018,10142018,10082018,10062018
Hectare Squirrel Number,5,4,7,3,8
Age,Adult,Adult,Adult,Adult,Adult
Primary Fur Color,Cinnamon,Gray,Gray,Gray,Gray
Highlight Fur Color,"Gray, White",Cinnamon,Cinnamon,NaN,Cinnamon


In [8]:
# Convert the date from %m%d%Y to pd.datetime.
squirrel['Date'] = pd.to_datetime(squirrel['Date'], format='%m%d%Y')

squirrel['Hectare'].nunique() #339/3023 unique values
squirrel['Hectare'] = squirrel['Hectare'].astype('category') # Convert object to category.

squirrel['Shift'].unique() # AM or PM
squirrel['Shift'] = squirrel['Shift'].astype('category') # Convert object to category.

squirrel['Hectare Squirrel Number'].max() # max value 23
squirrel['Hectare Squirrel Number'].min() # min value 1
squirrel['Hectare Squirrel Number'] = squirrel['Hectare Squirrel Number'].astype('int8') # Convert int64 to int8.

squirrel.Age.unique() # [nan, 'Adult', 'Juvenile', '?']
squirrel['Age'] = squirrel['Age'].astype('category') # Convert object to category.
squirrel['Age'] = squirrel['Age'].replace('?', np.nan) # treat ? as a missing value, NaN

squirrel['Primary Fur Color'].unique() # [nan, 'Gray', 'Cinnamon', 'Black']
squirrel['Primary Fur Color'] = squirrel['Primary Fur Color'].astype('category') # Convert object to category.

squirrel['Highlight Fur Color'].unique() # has multiple categories. e.g., 'Gray', 'Cinnamon, White', 'Gray, White'

# Step 1: Split the multiple categories into a list of categories
squirrel['Highlight Fur Color'] = squirrel['Highlight Fur Color'].str.split(', ')

# Step 2: Handle NaN values by filling them with an empty list
squirrel['Highlight Fur Color'] = squirrel['Highlight Fur Color'].apply(lambda x: x if isinstance(x, list) else [])

# Step 3: Explode the list to get each category in its own row
exploded = squirrel.explode('Highlight Fur Color')

# Step 4: Create dummy variables for each unique category
dummies = pd.get_dummies(exploded['Highlight Fur Color'], prefix='highlight_color')

# Step 5: Group by the original index and sum to collapse back into the original rows
dummies = dummies.groupby(exploded.index).sum()

# Step 6: Combine the original DataFrame with the dummies DataFrame
squirrel = pd.concat([squirrel, dummies], axis=1)

squirrel['Location'].unique() # [nan, 'Above Ground', 'Ground Plane']
squirrel['Location'] = squirrel['Location'].astype('category') # Convert object to category.

squirrel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 35 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   X                                           3023 non-null   float64       
 1   Y                                           3023 non-null   float64       
 2   Unique Squirrel ID                          3023 non-null   object        
 3   Hectare                                     3023 non-null   category      
 4   Shift                                       3023 non-null   category      
 5   Date                                        3023 non-null   datetime64[ns]
 6   Hectare Squirrel Number                     3023 non-null   int8          
 7   Age                                         2898 non-null   category      
 8   Primary Fur Color                           2968 non-null   category      
 9   Highligh

In [9]:
# Ordered and renamed columns

# Renaming columns in the DataFrame
renamed_columns = {
    'Unique Squirrel ID': 'unique_squirrel_id',
    'Hectare': 'hectare',
    'Hectare Squirrel Number': 'hectare_squirrel_number',
    'X': 'x',
    'Y': 'y',
    'Lat/Long': 'lat_long',
    'Date': 'date',
    'Shift': 'shift',
    'Age': 'age',
    'Primary Fur Color': 'primary_fur_color',
    'Highlight Fur Color': 'highlight_fur_color',
    'Combination of Primary and Highlight Color': 'combination_fur_color',
    'Color notes': 'color_notes',
    'Running': 'running',
    'Chasing': 'chasing',
    'Climbing': 'climbing',
    'Eating': 'eating',
    'Foraging': 'foraging',
    'Other Activities': 'other_activities',
    'Kuks': 'kuks',
    'Quaas': 'quaas',
    'Moans': 'moans',
    'Tail flags': 'tail_flags',
    'Tail twitches': 'tail_twitches',
    'Approaches': 'approaches',
    'Indifferent': 'indifferent',
    'Runs from': 'runs_from',
    'Other Interactions': 'other_interactions',
    'Location': 'location',
    'Above Ground Sighter Measurement': 'above_ground_sighter_measurement',
    'Specific Location': 'specific_location'
}

# Identification Columns
id_cols = [
    'unique_squirrel_id', 'hectare', 'hectare_squirrel_number'
]

# Location Columns
loc_cols = [
    'x', 'y', 'lat_long'
]

# Time Columns
time_cols = [
    'date', 'shift'
]

# Physical Characteristics
physical_char_cols = [
    'age', 'primary_fur_color', 'highlight_fur_color',
    'combination_fur_color', 'color_notes'
]

# Behavior Columns
behavior_cols = [
    'running', 'chasing', 'climbing', 'eating',
    'foraging', 'other_activities', 'kuks', 'quaas',
    'moans', 'tail_flags', 'tail_twitches', 'approaches',
    'indifferent', 'runs_from', 'other_interactions'
]

# Additional Location Details
other_loc_cols = [
    'location', 'above_ground_sighter_measurement', 'specific_location'
]

# Combined Ordered Columns
ordered_columns = (
    id_cols
    + loc_cols
    + time_cols
    + physical_char_cols
    + behavior_cols
    + other_loc_cols
)

# Applying the renaming and ordering to the DataFrame
squirrel = squirrel.rename(columns=renamed_columns)[ordered_columns]

In [10]:
squirrel.sample(n=5, random_state=28).T

,1945,1174,984,663,1757
unique_squirrel_id,5E-PM-1012-05,40D-PM-1013-04,22B-PM-1014-07,39D-AM-1008-03,31E-PM-1006-08
hectare,05E,40D,22B,39D,31E
hectare_squirrel_number,5,4,7,3,8
x,-73.974718,-73.955326,-73.968172,-73.956366,-73.960156
y,40.769282,40.797559,40.78356,40.796682,40.790051
lat_long,POINT (-73.9747175419546 40.7692820030878),POINT (-73.9553263156025 40.7975592060572),POINT (-73.9681722244801 40.783559826281),POINT (-73.9563661256875 40.7966823114508),POINT (-73.9601561064548 40.7900513933531)
date,2018-10-12 00:00:00,2018-10-13 00:00:00,2018-10-14 00:00:00,2018-10-08 00:00:00,2018-10-06 00:00:00
shift,PM,PM,PM,AM,PM
age,Adult,Adult,Adult,Adult,Adult
primary_fur_color,Cinnamon,Gray,Gray,Gray,Gray


In [11]:
squirrel.describe(include=['object', 'category']).T # there seems to be a duplicate in unique_squirrel_id. why?

,count,unique,top,freq
unique_squirrel_id,3023,3018,7D-PM-1010-01,2
hectare,3023,339,14D,32
lat_long,3023,3023,POINT (-73.9561344937861 40.7940823884086),1
shift,3023,2,PM,1676
age,2898,2,Adult,2568
primary_fur_color,2968,3,Gray,2473
highlight_fur_color,3023,11,[],1086
combination_fur_color,3023,22,Gray+,895
color_notes,182,135,Gray & Cinnamon selected as Primary. White sel...,9
other_activities,437,307,digging,19


In [12]:
squirrel.describe(include=['object', 'category']).T

,count,unique,top,freq
unique_squirrel_id,3023,3018,7D-PM-1010-01,2
hectare,3023,339,14D,32
lat_long,3023,3023,POINT (-73.9561344937861 40.7940823884086),1
shift,3023,2,PM,1676
age,2898,2,Adult,2568
primary_fur_color,2968,3,Gray,2473
highlight_fur_color,3023,11,[],1086
combination_fur_color,3023,22,Gray+,895
color_notes,182,135,Gray & Cinnamon selected as Primary. White sel...,9
other_activities,437,307,digging,19


In [13]:
squirrel.describe(include=['number'])

,hectare_squirrel_number,x,y
count,3023.000000,3023.000000,3023.000000
mean,4.123718,-73.967184,40.780853
std,3.096492,0.007726,0.010285
min,1.000000,-73.981159,40.764911
25%,2.000000,-73.973102,40.771676
50%,3.000000,-73.968594,40.778166
75%,6.000000,-73.960189,40.791219
max,23.000000,-73.949722,40.800119


In [14]:
squirrel.describe(include=['bool']).T

,count,unique,top,freq
running,3023,2,False,2293
chasing,3023,2,False,2744
climbing,3023,2,False,2365
eating,3023,2,False,2263
foraging,3023,2,False,1588
kuks,3023,2,False,2921
quaas,3023,2,False,2973
moans,3023,2,False,3020
tail_flags,3023,2,False,2868
tail_twitches,3023,2,False,2589


In [15]:
top_5_hectare = squirrel.groupby('hectare')['unique_squirrel_id'].count().sort_values(ascending=False).head(5)
top_5_hectare # further check what correlates with the squirrel sighting frequency in a hectare.

hectare
14D    32
32E    30
14E    28
01B    27
07H    26
Name: unique_squirrel_id, dtype: int64

In [16]:
top_primary_fur_color = squirrel.groupby('primary_fur_color')['unique_squirrel_id'].count().sort_values(ascending=False)
top_primary_fur_color # mostly gray. what's the temperature outside? is it the season that correlates with the color?

primary_fur_color
Gray        2473
Cinnamon     392
Black        103
Name: unique_squirrel_id, dtype: int64

In [17]:
behavior = squirrel[behavior_cols].select_dtypes('bool')
behavior.sum(axis=0).sort_values(ascending=False)

indifferent      1454
foraging         1435
eating            760
running           730
runs_from         678
climbing          658
tail_twitches     434
chasing           279
approaches        178
tail_flags        155
kuks              102
quaas              50
moans               3
dtype: int64

In [18]:
squirrel[id_cols + loc_cols].sort_values(by='hectare')

,unique_squirrel_id,hectare,hectare_squirrel_number,x,y,lat_long
1175,1A-PM-1014-04,01A,4,-73.980790,40.768216,POINT (-73.9807898224726 40.768215817511)
2935,1A-PM-1014-05,01A,5,-73.980988,40.768322,POINT (-73.9809876538513 40.7683223242143)
2490,1A-AM-1007-03,01A,3,-73.980691,40.768115,POINT (-73.9806905259671 40.7681152210858)
1054,1A-AM-1007-04,01A,4,-73.981108,40.767516,POINT (-73.9811078442462 40.767515940567)
2386,1A-PM-1014-01,01A,1,-73.980852,40.768315,POINT (-73.9808517324721 40.7683154647217)
...,...,...,...,...,...,...
3007,42H-PM-1014-04,42H,4,-73.950353,40.797654,POINT (-73.9503533343658 40.7976543603953)
2909,42H-PM-1014-03,42H,3,-73.950606,40.797619,POINT (-73.9506062997507 40.7976189496674)
574,42I-PM-1014-03,42I,3,-73.949722,40.796517,POINT (-73.9497217674555 40.796517007214)
2695,42I-PM-1014-01,42I,1,-73.950146,40.797094,POINT (-73.9501457233643 40.797094356558)


In [19]:
squirrel_count_per_hectare = squirrel.groupby('hectare')['unique_squirrel_id'].count().sort_values(ascending=False)
squirrel_count_per_hectare

hectare
14D    32
32E    30
14E    28
01B    27
07H    26
       ..
26B     1
18E     1
24I     1
24B     1
23D     1
Name: unique_squirrel_id, Length: 339, dtype: int64

In [20]:
squirrel_count_per_date = squirrel.groupby('date')['unique_squirrel_id'].count()
squirrel_count_per_date

date
2018-10-06    337
2018-10-07    405
2018-10-08    285
2018-10-10    335
2018-10-12    218
2018-10-13    434
2018-10-14    368
2018-10-17    216
2018-10-18    200
2018-10-19    158
2018-10-20     67
Name: unique_squirrel_id, dtype: int64

In [26]:
# Group by date and hectare, then count the unique squirrel IDs
squirrel_count_per_date_hectare = squirrel.groupby(['hectare', 'date'])['unique_squirrel_id'].count()

# Sort each group in descending order based on the count of unique squirrel IDs
squirrel_count_per_date_hectare_sorted = squirrel_count_per_date_hectare.groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(ascending=False))


# Display the sorted counts per date and hectare
#print(squirrel_count_per_date_hectare_sorted)

squirrel_count_per_date_hectare_sorted

hectare  date      
01A      2018-10-14    7
         2018-10-07    4
         2018-10-06    0
         2018-10-08    0
         2018-10-10    0
                      ..
42I      2018-10-13    0
         2018-10-17    0
         2018-10-18    0
         2018-10-19    0
         2018-10-20    0
Name: unique_squirrel_id, Length: 3729, dtype: int64